In [95]:
from pathlib import Path
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import soundfile as sf
import pandas as pd

import os 
import itertools
import re
import math
import csv
import random
from sklearn import tree
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn import linear_model
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.datasets import make_classification
from sklearn.ensemble import GradientBoostingClassifier
from scipy.fftpack import fftn, ifftn, fft, ifft
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from scipy import signal
import pickle
from sklearn.externals import joblib
from sklearn.datasets import load_digits
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit
import warnings

warnings.filterwarnings('ignore') 
#df = pd.read_csv("noAudible/Features_294too299hz.csv")
#df2 = pd.read_csv("noAudible/Features_10too15hz.csv")
df = pd.read_csv("Frequenzbänder/Features_290too300hz.csv")
#df4 = pd.read_csv("noAudible/Features_20too25hz.csv")
#df5 = pd.read_csv("noAudible/Features_25too30hz.csv")
#df = pd.read_csv("noAudible/Features_5too10hz.csv")

#frames =[df1,df2,df3,df4,df5]
#df = pd.concat(frames)



#df = pd.read_csv("Features_all_no_audible.csv")
#df = df[(df.modeID != 'audible')]
df

,channel_1_mav,channel_1_mav2,channel_1_rms,channel_1_iav,channel_1_ssi,channel_1_var,channel_1_wl,channel_1_iemg,channel_1_aac,channel_1_zc,...,channel_6_aac,channel_6_zc,channel_6_ssc,channel_6_wamp,channel_6_medf,channel_6_meanf,userID,sessionID,uttID,modeID
0,10.946000,7.340598,21.752917,3.665815e+04,1.584711e+06,4.731893e+02,7.324742e+04,3.665815e+04,21.871429,3346,...,9.345970,3346,3345,1335,6.306306,39.107933,1,1001,10010001,audible
1,0.086419,0.070751,0.116513,1.947884e+02,3.059890e+01,1.357537e-02,3.895292e+02,1.947884e+02,0.172817,2248,...,0.033495,2243,2242,0,12.053571,45.984699,1,1001,10010002,audible
2,19.296377,12.536715,42.820696,7.828540e+04,7.438964e+06,1.833611e+03,1.563333e+05,7.828540e+04,38.534207,4053,...,36.233828,4053,4052,1873,6.683168,38.304855,1,1001,10010003,audible
3,42.057715,32.880855,80.467505,1.751704e+05,2.696846e+07,6.475019e+03,3.502192e+05,1.751704e+05,84.086233,4161,...,83.396449,4161,4160,1987,15.180723,50.076907,1,1001,10010004,audible
4,407.264146,252.746993,1032.462140,1.914549e+06,5.011163e+09,1.065978e+06,3.824330e+06,1.914549e+06,813.514166,4697,...,43.953313,4696,4695,2010,15.351812,50.390344,1,1001,10010005,audible
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4795,0.051987,0.041133,0.073204,1.214414e+02,1.251817e+01,5.358803e-03,2.428110e+02,1.214414e+02,0.103943,2329,...,0.051254,2331,2330,0,1.293103,30.584603,8,8019,80190246,whisper
4796,0.025743,0.018776,0.036037,4.904114e+01,2.473926e+00,1.298648e-03,9.800980e+01,4.904114e+01,0.051449,1901,...,0.006337,1893,1892,0,4.761905,34.589674,8,8019,80190247,whisper
4797,0.028984,0.026187,0.037317,6.532896e+01,3.138866e+00,1.392576e-03,1.305891e+02,6.532896e+01,0.057937,2249,...,0.015839,2246,2245,0,1.339286,6.822201,8,8019,80190248,whisper
4798,0.006421,0.005012,0.008281,1.099229e+01,1.174135e-01,6.858261e-05,2.196567e+01,1.099229e+01,0.012830,1702,...,0.005699,1708,1707,0,5.294118,34.360480,8,8019,80190249,whisper


In [90]:
def split_data_per_session(session_id):
    temp_df = df[(df.sessionID == session_id)]
    data = temp_df.copy()
    data = shuffle(data)
    y = data['modeID']
    data.drop(labels=['userID','uttID', 'sessionID','modeID'], axis=1, inplace=True)
    return train_test_split(data, y, test_size=0.2)

def split_data_session(train_user_ids,test_user_ids):
    temp_df = df
    data = temp_df.copy()
    train_df = data.loc[data['sessionID'].isin(train_user_ids)]
    test_df = data.loc[data['sessionID'].isin(test_user_ids)]
    X_train = train_df.copy()
    X_test = test_df.copy()
    y_train = X_train['modeID']
    y_test = X_test['modeID']
    X_train.drop(labels=['userID','uttID', 'sessionID','modeID'], axis=1, inplace=True)
    X_test.drop(labels=['userID','uttID', 'sessionID','modeID'], axis=1, inplace=True)
    return X_train, X_test, y_train, y_test 

def split_data_utterance(train_user_ids,test_user_ids):
    temp_df = df
    data = temp_df.copy()
    train_df = data.loc[data['userID'].isin(train_user_ids)]
    test_df = data.loc[data['userID'].isin(test_user_ids)]
    X_train = train_df.copy()
    X_test = test_df.copy()
    y_train = X_train['modeID']
    y_test = X_test['modeID']
    X_train.drop(labels=['userID','uttID', 'sessionID','modeID'], axis=1, inplace=True)
    X_test.drop(labels=['userID','uttID', 'sessionID','modeID'], axis=1, inplace=True)
    return X_train, X_test, y_train, y_test 

def split_data(train_user_ids,test_user_ids):
    temp_df = df
    data = temp_df.copy()
    train_df = data.loc[data['userID'].isin(train_user_ids)]
    test_df = data.loc[data['userID'].isin(test_user_ids)]
    X_train = train_df.copy()
    X_test = test_df.copy()
    y_train = X_train['modeID']
    y_test = X_test['modeID']
    X_train.drop(labels=['userID','uttID', 'sessionID','modeID'], axis=1, inplace=True)
    X_test.drop(labels=['userID','uttID', 'sessionID','modeID'], axis=1, inplace=True)
    return X_train, X_test, y_train, y_test 

In [91]:
def lda_user1():
    train_ids = [2,3,4,5,6,7,8]
    test_ids = [1]
    X_train, X_test, y_train, y_test = split_data(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 19 is "+str(score))
    return score     
def lda_user2():
    train_ids = [1,3,4,5,6,7,8]
    test_ids = [2]
    X_train, X_test, y_train, y_test = split_data(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 19 is "+str(score))
    return score  
    
def lda_user3():
    train_ids = [1,2,4,5,6,7,8]
    test_ids = [3]
    X_train, X_test, y_train, y_test = split_data(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 19 is "+str(score))
    return score 
    
def lda_user4():
    train_ids = [1,2,3,5,6,7,8]
    test_ids = [4]
    X_train, X_test, y_train, y_test = split_data(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 19 is "+str(score))
    return score     
        
def lda_user5():
    train_ids = [1,2,3,4,6,7,8]
    test_ids = [5]
    X_train, X_test, y_train, y_test = split_data(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 19 is "+str(score))
    return score   
    
def lda_user6():
    train_ids = [1,2,3,4,5,7,8]
    test_ids = [6]
    X_train, X_test, y_train, y_test = split_data(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 19 is "+str(score))
    return score  
    
def lda_user7():
    train_ids = [1,2,3,4,5,6,8]
    test_ids = [7]
    X_train, X_test, y_train, y_test = split_data(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 19 is "+str(score))
    return score    
    
def lda_user8():
    train_ids = [1,2,3,4,5,6,7]
    test_ids = [8]
    X_train, X_test, y_train, y_test = split_data(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 19 is "+str(score))
    return score 

In [92]:
def lda_users():
    a = lda_user1()
    b = lda_user2()
    c = lda_user3()
    d = lda_user4()
    e = lda_user5()
    f = lda_user6()
    g = lda_user7()
    h = lda_user8()
    scores = np.array([a,b,c,d,e,f,g,h])
    return scores

In [93]:
def multiple():
    scores = np.array([])
    for i in range (31):
        test = lda_users()
        scores = np.concatenate([scores,test],axis=0)
    print("Durchschnitt:")
    print((scores.mean()))
    print(scores.std())
    

In [96]:
multiple()

Accuracy for session 19 is 0.3333333333333333
Accuracy for session 19 is 0.3328888888888889
Accuracy for session 19 is 0.3466666666666667
Accuracy for session 19 is 0.31
Accuracy for session 19 is 0.31333333333333335
Accuracy for session 19 is 0.36666666666666664
Accuracy for session 19 is 0.33
Accuracy for session 19 is 0.31523809523809526
Accuracy for session 19 is 0.3333333333333333
Accuracy for session 19 is 0.3328888888888889
Accuracy for session 19 is 0.3466666666666667
Accuracy for session 19 is 0.31
Accuracy for session 19 is 0.31333333333333335
Accuracy for session 19 is 0.36666666666666664
Accuracy for session 19 is 0.33
Accuracy for session 19 is 0.31523809523809526
Accuracy for session 19 is 0.3333333333333333
Accuracy for session 19 is 0.3328888888888889
Accuracy for session 19 is 0.3466666666666667
Accuracy for session 19 is 0.31
Accuracy for session 19 is 0.31333333333333335
Accuracy for session 19 is 0.36666666666666664
Accuracy for session 19 is 0.33
Accuracy for sessi

Accuracy for session 19 is 0.3328888888888889
Accuracy for session 19 is 0.3466666666666667
Accuracy for session 19 is 0.31
Accuracy for session 19 is 0.31333333333333335
Accuracy for session 19 is 0.36666666666666664
Accuracy for session 19 is 0.33
Accuracy for session 19 is 0.31523809523809526
Accuracy for session 19 is 0.3333333333333333
Accuracy for session 19 is 0.3328888888888889
Accuracy for session 19 is 0.3466666666666667
Accuracy for session 19 is 0.31
Accuracy for session 19 is 0.31333333333333335
Accuracy for session 19 is 0.36666666666666664
Accuracy for session 19 is 0.33
Accuracy for session 19 is 0.31523809523809526
Accuracy for session 19 is 0.3333333333333333
Accuracy for session 19 is 0.3328888888888889
Accuracy for session 19 is 0.3466666666666667
Accuracy for session 19 is 0.31
Accuracy for session 19 is 0.31333333333333335
Accuracy for session 19 is 0.36666666666666664
Accuracy for session 19 is 0.33
Accuracy for session 19 is 0.31523809523809526
Accuracy for sess